In [211]:

import pandas as pd


df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv', encoding='latin1')

df.head(1)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [212]:
df.shape

(7043, 21)

In [213]:
df.info

<bound method DataFrame.info of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL 

In [214]:
df['Churn'].value_counts(normalize=True)
#هان وضحت نسبة العملاء المتسربين من الشركة 

Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

In [215]:
df['Churn'].value_counts(normalize=True) * 100

Churn
No     73.463013
Yes    26.536987
Name: proportion, dtype: float64

In [216]:
df['Churn'].value_counts()

Churn
No     5174
Yes    1869
Name: count, dtype: int64

In [217]:
df['Contract'].value_counts()

Contract
Month-to-month    3875
Two year          1695
One year          1473
Name: count, dtype: int64

In [218]:
# مقارنة نوع العقد بمعدل التسرب
contract_churn = pd.crosstab(df['Contract'], df['Churn'], normalize='index') * 100
print(contract_churn)

Churn                  No        Yes
Contract                            
Month-to-month  57.290323  42.709677
One year        88.730482  11.269518
Two year        97.168142   2.831858


In [219]:
contract_churn = pd.crosstab(df['PaymentMethod'], df['Churn'], normalize='index') * 100
print(contract_churn)

Churn                             No        Yes
PaymentMethod                                  
Bank transfer (automatic)  83.290155  16.709845
Credit card (automatic)    84.756899  15.243101
Electronic check           54.714588  45.285412
Mailed check               80.893300  19.106700


In [220]:
df.dtypes

customerID              str
gender                  str
SeniorCitizen         int64
Partner                 str
Dependents              str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
PaperlessBilling        str
PaymentMethod           str
MonthlyCharges      float64
TotalCharges            str
Churn                   str
dtype: object

In [221]:
import pandas as pd
import numpy as np

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# --- (Missing Values) ---
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100
missing_report = pd.concat([missing_count, missing_percentage], axis=1, keys=['Count', 'Percentage (%)'])
print("--- Missing Values Report ---")
print(missing_report[missing_report['Count'] > 0]) # عرض المفقود فقط
print("\n")

# --- (Duplicates) ---
customer_duplicates = df['customerID'].duplicated().sum()
print(f"--- Duplicates Check ---")
print(f"Number of duplicate customerIDs: {customer_duplicates}")
print("\n")

# --(Invalid Values / Empty Spaces) ---
# فحص المسافات الفارغة في عمود TotalCharges الذي اكتشفناه سابقاً
empty_spaces = (df['TotalCharges'] == " ").sum()
print(f"--- Invalid Values Check ---")
print(f"Empty spaces in TotalCharges: {empty_spaces}")
# فحص الفئات الغريبة في عمود Churn (تأكد من وجود Yes/No فقط)
print(f"Unique values in Churn: {df['Churn'].unique()}")
print("\n")

# ---(Outliers) ---
# ملاحظة: يجب تحويل TotalCharges لرقم أولاً ليعمل الفحص
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

def get_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] < lower_bound) | (df[column] > upper_bound)]

monthly_outliers = get_outliers(df, 'MonthlyCharges')
total_outliers = get_outliers(df.dropna(subset=['TotalCharges']), 'TotalCharges')

print(f"--- Outliers Report ---")
print(f"Outliers in MonthlyCharges: {len(monthly_outliers)}")
print(f"Outliers in TotalCharges: {len(total_outliers)}")

--- Missing Values Report ---
Empty DataFrame
Columns: [Count, Percentage (%)]
Index: []


--- Duplicates Check ---
Number of duplicate customerIDs: 0


--- Invalid Values Check ---
Empty spaces in TotalCharges: 11
Unique values in Churn: <StringArray>
['No', 'Yes']
Length: 2, dtype: str


--- Outliers Report ---
Outliers in MonthlyCharges: 0
Outliers in TotalCharges: 0


In [222]:
# TotalCharges: "It contains 11 spaces, which caused it to be identified as a string and hindered calculations."

# customerID: "There are no duplicates, but it remains a unique identifier and is not used in statistical analysis."

# Outliers: "There are no outliers, meaning the financial data is consistent and follows a normal pattern."

In [223]:
# تحويل عمود SeniorCitizen من Integer إلى Boolean
df['SeniorCitizen'] = df['SeniorCitizen'].astype(bool)

# للتأكد من النتيجة
print(df['SeniorCitizen'].head())
print(df['SeniorCitizen'].dtype)

0    False
1    False
2    False
3    False
4    False
Name: SeniorCitizen, dtype: bool
bool


In [224]:
# تحويل عمود SeniorCitizen من Integer إلى Boolean
df['Partner'] = df['Partner'].astype(bool)

# للتأكد من النتيجة
print(df['Partner'].head())
print(df['Partner'].dtype)

0    True
1    True
2    True
3    True
4    True
Name: Partner, dtype: bool
bool


In [225]:

df['Dependents'] = df['Dependents'].astype(bool)

print(df['Dependents'].head())
print(df['Dependents'].dtype)

0    True
1    True
2    True
3    True
4    True
Name: Dependents, dtype: bool
bool


In [226]:

df['PhoneService'] = df['PhoneService'].astype(bool)

print(df['PhoneService'].head())
print(df['PhoneService'].dtype)

0    True
1    True
2    True
3    True
4    True
Name: PhoneService, dtype: bool
bool


In [227]:

df['MultipleLines'] = df['MultipleLines'].astype(bool)

print(df['MultipleLines'].head())
print(df['MultipleLines'].dtype)

0    True
1    True
2    True
3    True
4    True
Name: MultipleLines, dtype: bool
bool


In [228]:

df['OnlineSecurity'] = df['OnlineSecurity'].astype(bool)


# للتأكد من النتيجة
print(df['OnlineSecurity'].head())
print(df['OnlineSecurity'].dtype)

df['DeviceProtection'] = df['DeviceProtection'].astype(bool)
# للتأكد من النتيجة
print(df['DeviceProtection'].head())
print(df['DeviceProtection'].dtype)

df['TechSupport'] = df['TechSupport'].astype(bool)
# للتأكد من النتيجة
print(df['TechSupport'].head())
print(df['TechSupport'].dtype)

df['PaperlessBilling'] = df['PaperlessBilling'].astype(bool)
# للتأكد من النتيجة
print(df['PaperlessBilling'].head())
print(df['PaperlessBilling'].dtype)

df['StreamingTV'] = df['StreamingTV'].astype(bool)
# للتأكد من النتيجة
print(df['StreamingTV'].head())
print(df['StreamingTV'].dtype)
df['OnlineBackup'] = df['OnlineBackup'].astype(bool)
# للتأكد من النتيجة
print(df['OnlineBackup'].head())
print(df['OnlineBackup'].dtype)
#كملي باقي 


0    True
1    True
2    True
3    True
4    True
Name: OnlineSecurity, dtype: bool
bool
0    True
1    True
2    True
3    True
4    True
Name: DeviceProtection, dtype: bool
bool
0    True
1    True
2    True
3    True
4    True
Name: TechSupport, dtype: bool
bool
0    True
1    True
2    True
3    True
4    True
Name: PaperlessBilling, dtype: bool
bool
0    True
1    True
2    True
3    True
4    True
Name: StreamingTV, dtype: bool
bool
0    True
1    True
2    True
3    True
4    True
Name: OnlineBackup, dtype: bool
bool


In [229]:
import pandas as pd

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


df['TotalCharges'] = df['TotalCharges'].fillna(0)

# سنقوم بربط 'Yes' بالرقم 1 و 'No' بالرقم 0
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# تحويل نوع العمود إلى integer لضمان التنسيق الرقمي
df['Churn'] = df['Churn'].astype(int)
# التأكد من النتائج بعد الإصلاح
print("أنواع البيانات بعد التحويل:")
print(df[['TotalCharges', 'Churn']].dtypes) 

print("\nأول 5 صفوف من النتائج:")
print(df[['TotalCharges', 'Churn']].head())



أنواع البيانات بعد التحويل:
TotalCharges    float64
Churn             int64
dtype: object

أول 5 صفوف من النتائج:
   TotalCharges  Churn
0         29.85      0
1       1889.50      0
2        108.15      1
3       1840.75      0
4        151.65      1


In [230]:
#هنا لملء البيانات الفارغة 
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)

df['TotalCharges'].isnull().sum()

df['TotalCharges'].dtype

dtype('float64')

In [231]:
# قائمة الأعمدة التي تحتوي على "No internet service"
cols_to_fix = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                'TechSupport', 'StreamingTV', 'StreamingMovies']

# استبدال "No internet service" بـ "No" لتوحيد الفئات
for col in cols_to_fix:
    df[col] = df[col].replace('No internet service', 'No')

# إصلاح عمود خطوط الهاتف أيضاً
df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

# التأكد من النتائج
print("الفئات الجديدة في عمود OnlineSecurity:")
print(df['OnlineSecurity'].unique())

الفئات الجديدة في عمود OnlineSecurity:
[ True]


In [232]:
df.drop('customerID', axis=1, inplace=True)



In [233]:
import pandas as pd


df['is_month_to_month'] = df['Contract'].apply(lambda x: 1 if x == 'Month-to-month' else 0)#lamda هي دالة سريعة للمرور على كل العمود
def categorize_contract(contract):
    if contract == 'Month-to-month':
        return 'month'
    elif contract == 'One year':
        return 'year'
    elif contract == 'Two year':
        return 'two-year'
    else:
        return 'unknown'

df['contract_length_group'] = df['Contract'].apply(categorize_contract)

# عرض أول 5 صفوف للتأكد من النتيجة
print(df[['Contract', 'is_month_to_month', 'contract_length_group']].head())

         Contract  is_month_to_month contract_length_group
0  Month-to-month                  1                 month
1        One year                  0                  year
2  Month-to-month                  1                 month
3        One year                  0                  year
4  Month-to-month                  1                 month


In [234]:

#اضفت 1 لتجنب القسمة على 0
df['avg_spend_per_month'] = df['TotalCharges'] / (df['tenure'] + 1)

# نقارن قيمة كل صف بالوسيط (median) لعمود MonthlyChargesلتحديد العملاء مرتفعين الدفع 
monthly_median = df['MonthlyCharges'].median()
df['high_monthly_charge'] = (df['MonthlyCharges'] > monthly_median).astype(int)

total_median = df['TotalCharges'].median()
df['high_total_charge'] = (df['TotalCharges'] > total_median).astype(int)

print(df[['tenure', 'avg_spend_per_month', 'high_monthly_charge', 'high_total_charge']].head())

   tenure  avg_spend_per_month  high_monthly_charge  high_total_charge
0       1            14.925000                    0                  0
1      34            53.985714                    0                  1
2       2            36.050000                    0                  0
3      45            40.016304                    0                  1
4       2            50.550000                    1                  0


In [235]:
#نلاحظ ان العميل رقم 4 يدفع اكثر من متوسط وهنا هو عرضة لتسريب 

In [236]:
# 1. تحديد العملاء الجدد (فترة بقاء 3 أشهر أو أقل)
# سنحول النتيجة إلى (0 و 1) ليسهل معالجتها رياضياً
df['new_customer'] = (df['tenure'] <= 3).astype(int)

# 2. تحديد العملاء القدامى)
df['long_term_customer'] = (df['tenure'] >= 24).astype(int)

# عرض عينة من البيانات للتحقق
print(df[['tenure', 'new_customer', 'long_term_customer']].head(10))

   tenure  new_customer  long_term_customer
0       1             1                   0
1      34             0                   1
2       2             1                   0
3      45             0                   1
4       2             1                   0
5       8             0                   0
6      22             0                   0
7      10             0                   0
8      28             0                   1
9      62             0                   1


In [237]:
# 1. تحديد مستخدمي "الشيك الإلكتروني"
df['uses_electronic_check'] = (df['PaymentMethod'] == 'Electronic check').astype(int)

# 2. تحويل "الفواتير الورقية" إلى قيم رقمية (0 أو 1)
# إذا كان العمود الأصلي يحتوي على 'Yes' و 'No'
df['paperless_billing'] = df['PaperlessBilling'].apply(lambda x: 1 if x == 'Yes' else 0)

# عرض النتيجة للتأكد
print(df[['PaymentMethod', 'uses_electronic_check', 'PaperlessBilling', 'paperless_billing']].head())

               PaymentMethod  uses_electronic_check  PaperlessBilling  \
0           Electronic check                      1              True   
1               Mailed check                      0              True   
2               Mailed check                      0              True   
3  Bank transfer (automatic)                      0              True   
4           Electronic check                      1              True   

   paperless_billing  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  


In [238]:
# 1. إنشاء ميزة: العميل لا يملك حماية إلكترونية
# بما أن العمود Boolean، فإن ~ تعكس القيمة (True تصبح False والعكس)
# ثم نحولها إلى 1 (خطر) و 0 (أمان)
df['no_online_security'] = (~df['OnlineSecurity']).astype(int)

# 2. تحليل إضافي: هل العميل جديد وبدون حماية؟ 
# (هذا المزيج هو الأكثر عرضة للتسرب)
if 'new_customer' in df.columns:
    df['unprotected_new_customer'] = ((df['no_online_security'] == 1) & (df['new_customer'] == 1)).astype(int)

# عرض النتيجة للتأكد من الميزات الجديدة
print(df[['OnlineSecurity', 'no_online_security']].head())

   OnlineSecurity  no_online_security
0            True                   0
1            True                   0
2            True                   0
3            True                   0
4            True                   0


In [239]:
# 1. تحويل StreamingTV إلى ميزة رقمية
# نضع 1 إذا كان مشتركاً و 0 إذا لم يكن
df['uses_streaming_tv'] = (df['StreamingTV'] == 'Yes').astype(int)

# 2. إنشاء ميزة "العميل الملتصق" (Sticky Customer)
# العميل الذي لديه حماية أمنية (العمود البولين لديك) وخدمة تلفزيون
# هؤلاء هم الأقل عرضة للمغادرة
df['sticky_customer'] = ((df['OnlineSecurity'] == True) & (df['uses_streaming_tv'] == 1)).astype(int)

# 3. ميزة "الترفيه فقط" (Entertainment Only)
# إذا كان لديه تلفزيون لكن ليس لديه حماية أمنية، قد يكون عميلاً غير مستقر
df['entertainment_no_security'] = ((df['OnlineSecurity'] == False) & (df['uses_streaming_tv'] == 1)).astype(int)
#

In [240]:
# 1. تحديد عملاء الألياف الضوئية (غالباً هم الأكثر عرضة للمغادرة بسبب السعر)
df['is_fiber_optic'] = (df['InternetService'] == 'Fiber optic').astype(int)

# 2. تحديد العملاء بدون إنترنت (الأكثر وفاءً/أقل خطورة)
df['no_internet'] = (df['InternetService'] == 'No').astype(int)

# 3. ميزة "الخطر المالي والتقني": عميل ألياف ضوئية وبدون حماية أمنية
# (بما أن OnlineSecurity لديك Boolean كما ذكرت سابقاً)
df['fiber_no_security'] = ((df['is_fiber_optic'] == 1) & (df['OnlineSecurity'] == False)).astype(int)

In [241]:
#ند إدخال InternetService في نموذج الذكاء الاصطناعي، سيتعلم الموديل أن:

#Fiber optic + High Monthly Charges = احتمالية تسرب مرتفعة.

#DSL + Long Tenure = احتمالية تسرب منخفضة جداً.

#No Internet Service = عميل مستقر للغاية ولكنه قليل الربحية
#استعنت في الشات لاستنتاج العلاقات 

In [242]:
# 1. تحويل الدعم الفني إلى ميزة رقمية
# 1 إذا كان لديه دعم، 0 إذا لم يكن لديه 
df['has_tech_support'] = (df['TechSupport'] == 'Yes').astype(int)



if 'is_fiber_optic' in df.columns:
    df['vulnerable_fiber_user'] = ((df['is_fiber_optic'] == 1) & (df['has_tech_support'] == 0)).astype(int)

# عميل لديه حماية أمنية (العمود البولين لديك) ودعم فني
df['fully_secured_user'] = ((df['OnlineSecurity'] == True) & (df['has_tech_support'] == 1)).astype(int)

In [243]:
# 1. تحويل تعدد الخطوط إلى ميزة رقمية (1 للمشترك، 0 لغير ذلك)
df['has_multiple_lines'] = (df['MultipleLines'] == 'Yes').astype(int)

# 2.  (High-Value Telephony)
# عميل لديه خطوط متعددة وفاتورة شهرية عالية
if 'high_monthly_charge' in df.columns:
    df['premium_phone_user'] = ((df['has_multiple_lines'] == 1) & (df['high_monthly_charge'] == 1)).astype(int)

# 3. ميزة "الارتباط الكامل": إنترنت ألياف + خطوط متعددة
if 'is_fiber_optic' in df.columns:
    df['full_home_package'] = ((df['is_fiber_optic'] == 1) & (df['has_multiple_lines'] == 1)).astype(int)

In [244]:

df.info()

if df.isnull().sum().sum() == 0:
    print("\n✅ لا توجد قيم مفقودة.")
else:
    print(f"\n⚠️ عدد القيم المفقودة: {df.isnull().sum().sum()}")

print(f"\n📏 حجم البيانات الحالي (Rows, Columns): {df.shape}")

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   gender                     7043 non-null   str    
 1   SeniorCitizen              7043 non-null   bool   
 2   Partner                    7043 non-null   bool   
 3   Dependents                 7043 non-null   bool   
 4   tenure                     7043 non-null   int64  
 5   PhoneService               7043 non-null   bool   
 6   MultipleLines              7043 non-null   bool   
 7   InternetService            7043 non-null   str    
 8   OnlineSecurity             7043 non-null   bool   
 9   OnlineBackup               7043 non-null   bool   
 10  DeviceProtection           7043 non-null   bool   
 11  TechSupport                7043 non-null   bool   
 12  StreamingTV                7043 non-null   bool   
 13  StreamingMovies            7043 non-null   str    
 14  Con

In [245]:

original_cols = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 
                 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 
                 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 
                 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']

current_cols = df.columns.tolist()

added_features = [col for col in current_cols if col not in original_cols]

dropped_cols = [col for col in original_cols if col not in current_cols]


print("📊 --- تقرير هيكلية البيانات النهائي --- 📊")
print(f"\n✅ الميزات الجديدة المضافة ({len(added_features)}):")
for feature in added_features:
    print(f"   - {feature}")

print(f"\n🗑️  الأعمدة التي تم حذفها أو استبدالها ({len(dropped_cols)}):")
for col in dropped_cols:
    print(f"   - {col}")

print(f"\n📏 الحجم النهائي للبيانات: {df.shape}")

📊 --- تقرير هيكلية البيانات النهائي --- 📊

✅ الميزات الجديدة المضافة (24):
   - Churn
   - is_month_to_month
   - contract_length_group
   - avg_spend_per_month
   - high_monthly_charge
   - high_total_charge
   - new_customer
   - long_term_customer
   - uses_electronic_check
   - paperless_billing
   - no_online_security
   - unprotected_new_customer
   - uses_streaming_tv
   - sticky_customer
   - entertainment_no_security
   - is_fiber_optic
   - no_internet
   - fiber_no_security
   - has_tech_support
   - vulnerable_fiber_user
   - fully_secured_user
   - has_multiple_lines
   - premium_phone_user
   - full_home_package

🗑️  الأعمدة التي تم حذفها أو استبدالها (1):
   - customerID

📏 الحجم النهائي للبيانات: (7043, 43)


In [246]:
df.head(20)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,entertainment_no_security,is_fiber_optic,no_internet,fiber_no_security,has_tech_support,vulnerable_fiber_user,fully_secured_user,has_multiple_lines,premium_phone_user,full_home_package
0,Female,False,True,True,1,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
1,Male,False,True,True,34,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
2,Male,False,True,True,2,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
3,Male,False,True,True,45,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
4,Female,False,True,True,2,True,True,Fiber optic,True,True,...,0,1,0,0,0,1,0,0,0,0
5,Female,False,True,True,8,True,True,Fiber optic,True,True,...,0,1,0,0,0,1,0,0,0,0
6,Male,False,True,True,22,True,True,Fiber optic,True,True,...,0,1,0,0,0,1,0,0,0,0
7,Female,False,True,True,10,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
8,Female,False,True,True,28,True,True,Fiber optic,True,True,...,0,1,0,0,0,1,0,0,0,0
9,Male,False,True,True,62,True,True,DSL,True,True,...,0,0,0,0,0,0,0,0,0,0
